In [35]:
import cv2 as cv
import numpy as np

In [36]:
# Reading images
img_path = "../images/"
img_1_name = "out_2.jpg"
img_2_name = "out_3.jpg"
img_3_name = "out_4.jpg"
img_1 = cv.imread(img_path + img_1_name, cv.IMREAD_COLOR)
img_2 = cv.imread(img_path + img_2_name, cv.IMREAD_COLOR)
img_3 = cv.imread(img_path + img_3_name, cv.IMREAD_COLOR)

In [37]:
# Detect feature points and compute descriptors
sift = cv.SIFT_create()

img_1_fp, img_1_des = sift.detectAndCompute(img_1, None)
img_2_fp, img_2_des = sift.detectAndCompute(img_2, None)
img_3_fp, img_3_des = sift.detectAndCompute(img_3, None)

In [38]:
# Displaying SIFT feature points with scale and orientation
img_1_out = cv.drawKeypoints(img_1, img_1_fp, None, color = (255, 0, 0) , flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img_2_out = cv.drawKeypoints(img_2, img_2_fp, None, color = (255, 0, 0) , flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img_3_out = cv.drawKeypoints(img_3, img_3_fp, None, color = (255, 0, 0) , flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

img_out = np.concatenate((img_1_out, img_2_out, img_3_out), axis=1)

cv.imwrite(img_path + "panorama_features.jpg", img_out)

True

In [39]:
# Creating brute force descriptor matcher
matcher = cv.BFMatcher(crossCheck=False)

In [40]:
def find_best_matches(query_des, train_des, k, knn_ratio):
    # Find kNN matches with k = 2
    matches = matcher.knnMatch(query_des, train_des, k=k)
    # Select good matches
    good = []
    for m in matches:
        if len(m) > 1:
            if m[0].distance < knn_ratio * m[1].distance:
                good.append(m[0])
    return good

In [41]:
# Finding k-nearest best match for 1-2 and 3-2 descriptors of images and filtering them
matches_1_2 = find_best_matches(img_1_des, img_2_des, 2, 0.75)
matches_3_2 = find_best_matches(img_3_des, img_2_des, 2, 0.75)

In [42]:
# Displaying top 50 matches
num_matches = 50
matches_1_2 = sorted(matches_1_2, key = lambda x:x.distance)
matches_3_2 = sorted(matches_3_2, key = lambda x:x.distance)

img_match_1_2 = cv.drawMatches(img_1, img_1_fp, img_2, img_2_fp, matches_1_2[:num_matches], None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS, matchColor=(255, 0, 0))
img_match_3_2 = cv.drawMatches(img_3, img_3_fp, img_2, img_2_fp, matches_3_2[:num_matches], None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS, matchColor=(255, 0, 0))

cv.imwrite(img_path + "matches_1_2.jpg", img_match_1_2)
cv.imwrite(img_path + "matches_3_2.jpg", img_match_3_2)

True

In [43]:
# Executing RANSAC to calculate the transformation matrix
MIN_MATCH_COUNT = 10
if len(matches_1_2) < MIN_MATCH_COUNT or len(matches_3_2) < MIN_MATCH_COUNT:
    print("Not enough matches.")

# Create arrays of point coordinates
img_1_pts = np.float32([img_1_fp[m.queryIdx].pt for m in matches_1_2]).reshape(-1, 1, 2)
img_2_1_pts = np.float32([img_2_fp[m.trainIdx].pt for m in matches_1_2]).reshape(-1, 1, 2)

img_3_pts = np.float32([img_3_fp[m.queryIdx].pt for m in matches_3_2]).reshape(-1, 1, 2)
img_2_3_pts = np.float32([img_2_fp[m.trainIdx].pt for m in matches_3_2]).reshape(-1, 1, 2)

# Run RANSAC method
M_1_2, mask_1_2 = cv.findHomography(img_1_pts, img_2_1_pts, cv.RANSAC, 5)
mask_1_2 = mask_1_2.ravel().tolist()

M_3_2, mask_3_2 = cv.findHomography(img_3_pts, img_2_3_pts, cv.RANSAC, 5)
mask_3_2 = mask_3_2.ravel().tolist()

In [44]:
# Displaying inlier matches
img_trans_1_2 = cv.drawMatches(img_1, img_1_fp, img_2, img_2_fp, matches_1_2, None, matchesMask=mask_1_2, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS, matchColor=(0, 255, 0))
img_trans_3_2 = cv.drawMatches(img_3, img_3_fp, img_2, img_2_fp, matches_3_2, None, matchesMask=mask_3_2, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS, matchColor=(0, 255, 0))

cv.imwrite(img_path + "img_inliers_1_2.jpg", img_trans_1_2)
cv.imwrite(img_path + "img_inliers_3_2.jpg", img_trans_3_2)

True

In [45]:
# Images corners
h_1, w_1 = img_1.shape[:2]
h_3, w_3 = img_3.shape[:2]
img_1_box = np.float32([[0, 0], [0, h_1 - 1], [w_1 - 1, h_1 - 1], [w_1 - 1, 0]]).reshape(-1, 1, 2)
img_3_box = np.float32([[0, 0], [0, h_3 - 1], [w_3 - 1, h_3 - 1], [w_3 - 1, 0]]).reshape(-1, 1, 2)
img_1_to_img_2_box =cv.perspectiveTransform(img_1_box, M_1_2)
img_3_to_img_2_box =cv.perspectiveTransform(img_3_box, M_3_2)

In [46]:
# Compute paddings
top_padding = int(max(-img_1_to_img_2_box[0][0, 1], -img_3_to_img_2_box[3][0, 1], 0))
bot_padding = int(max(img_1_to_img_2_box[1][0, 1] - img_2.shape[0], img_3_to_img_2_box[2][0, 1] - img_2.shape[0], 0))
left_padding = int(max(-img_1_to_img_2_box[0][0, 0], -img_1_to_img_2_box[1][0, 0], 0))
right_padding = int(max(img_3_to_img_2_box[3][0, 0] - img_2.shape[1], img_3_to_img_2_box[2][0, 0] - img_2.shape[1], 0))

In [47]:
# Create panoramic image
panoramic_img = cv.copyMakeBorder(img_2, top_padding, bot_padding, left_padding, right_padding, borderType=cv.BORDER_CONSTANT)
cv.imwrite(img_path + "panoramic_img.jpg", panoramic_img)

True

In [48]:
# Shift images boxes
translation_matrix = np.array([[1, 0, left_padding], [0, 1, top_padding], [0, 0, 1]])
img_1_to_img_2_box_shifted = cv.perspectiveTransform(img_1_to_img_2_box, translation_matrix)
img_3_to_img_2_box_shifted = cv.perspectiveTransform(img_3_to_img_2_box, translation_matrix)

In [49]:
# Draw boxes on panoramic image
pano_with_boxes = np.copy(panoramic_img)
pano_with_boxes = cv.polylines(pano_with_boxes, [np.int32(img_1_to_img_2_box_shifted)], True, (255, 0, 0), 3, cv.LINE_AA)
pano_with_boxes = cv.polylines(pano_with_boxes, [np.int32(img_3_to_img_2_box_shifted)], True, (255, 0, 0), 3, cv.LINE_AA)
cv.imwrite(img_path + "pano_boxes.jpg", pano_with_boxes)

True

In [50]:
# Perform warp for images 1 and 3
img_1_warp =cv.warpPerspective(img_1, np.matmul(translation_matrix, M_1_2), dsize=(panoramic_img.shape[1], panoramic_img.shape[0]))
img_3_warp =cv.warpPerspective(img_3, np.matmul(translation_matrix, M_3_2), dsize=(panoramic_img.shape[1], panoramic_img.shape[0]))
cv.imwrite(img_path + "img_1_warp.jpg", img_1_warp)
cv.imwrite(img_path + "img_3_warp.jpg", img_3_warp)

True

In [51]:
# Create masks for images 1, 2, 3
pan_h, pan_w = panoramic_img.shape[:2]

img_1_mask = np.zeros((pan_h, pan_w), dtype=np.uint8)
img_1_mask = cv.fillConvexPoly(img_1_mask, np.int32(img_1_to_img_2_box_shifted), 255)

img_2_mask = np.zeros((pan_h, pan_w), dtype=np.uint8)
img_2_mask[top_padding:(pan_h - bot_padding), left_padding:(pan_w - right_padding)] = 255

img_3_mask = np.zeros((pan_h, pan_w), dtype=np.uint8)
img_3_mask = cv.fillConvexPoly(img_3_mask, np.int32(img_3_to_img_2_box_shifted), 255)

# Save masks
cv.imwrite(img_path + "img_1_mask.jpg", img_1_mask)
cv.imwrite(img_path + "img_2_mask.jpg", img_2_mask)
cv.imwrite(img_path + "img_3_mask.jpg", img_3_mask)

True

In [52]:
# Marge all images into panoramic one

img_1_3_mask = cv.bitwise_xor(img_1_mask, img_3_mask) - cv.bitwise_and(cv.bitwise_xor(img_1_mask, img_3_mask), img_2_mask)

img_1_warp_cropped = cv.bitwise_and(img_1_warp, img_1_warp, mask=img_1_3_mask)
img_3_warp_cropped = cv.bitwise_and(img_3_warp, img_3_warp, mask=img_1_3_mask)

result = cv.add(cv.add(img_1_warp_cropped, img_3_warp_cropped), panoramic_img)

cv.imwrite(img_path + "result.jpg", result)

True